## Query 10000 Playlists for Final Model

In [1]:
#importing packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
playlist = pd.read_csv("../raw_data/MPD_Playlist.csv", index_col="Unnamed: 0") #read master playlist data 
track_info = pd.read_csv("../raw_data/track_info.csv", index_col="Unnamed: 0") #read master track info data
artist_info = pd.read_csv("../raw_data/artist_info.csv", index_col="Unnamed: 0") #read master artist info data

/Users/KaReeNaaah/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
playlist.head()

,Playlistid,Playlist,Album,Track,Artist,Edit,Follower,Collaborative,MPDid
0,549000,Bob Dylan,65,75,39,28,1,False,549000
1,549001,ON THE RUN,30,61,5,4,1,False,549001
2,549002,Biking,117,144,72,12,1,False,549002
3,549003,August,32,38,33,20,1,False,549003
4,549004,run mix,15,16,12,3,1,False,549004


In [4]:
playlist_filter = playlist[playlist.Track >= 10]

In [5]:
#select random 10,000 playlist ids from playlist
np.random.seed(123) #set seed
rand10k = np.random.choice(playlist_filter.Playlistid, size = 10000, replace = False)

In [6]:
track_info.shape #check shape of data

(2399969, 15)

In [7]:
artist_info.shape #check shape of data

(295800, 4)

In [8]:
#extract track info from randomly selected playlistid
import os
track_data = list()
directory = '../raw_data/data_Track/'
for file in os.listdir(directory):
    if '.csv' in file:
        try:
            df = pd.read_csv(directory+file, index_col = "Unnamed: 0")
            df = df[df['Playlistid'].isin(rand10k)] #only extract playlistids that were randomly selected
            track_data.append(df) #append to list
        except:
            track_data = track_data

In [9]:
rand10k_track = pd.concat(track_data) #concatenate data frames

In [10]:
rand10k_track.shape #check shape of concatenated data

(679054, 9)

In [11]:
track_info = track_info.drop('id', axis = 1).rename(columns = {'uri': 'Track_uri'}) #rename cols for merging

In [12]:
subset10k = rand10k_track.merge(track_info, on = 'Track_uri', how = 'left') #add track info to subset10k

In [13]:
subset10k = subset10k.drop_duplicates() #drop duplicates

In [14]:
subset10k.shape #check shape of subset data

(679054, 22)

In [15]:
subset10k.isna().sum() #check missing data

Playlistid               0
Trackid                  0
Artist_Name              0
Track_Name               0
Album_Name               0
Track_Duration           0
Artist_uri               0
Track_uri                0
Album_uri                0
danceability        526167
energy              526167
key                 526167
loudness            526167
mode                526167
speechiness         526167
acousticness        526167
instrumentalness    526167
liveness            526167
valence             526167
tempo               526167
duration_ms         526167
time_signature      526167
dtype: int64

In [ ]:
# missing = subset10k[subset10k['danceability'].isna()] 

# missing_uri = missing.Track_uri.unique() #generate list of missing uri for query

# missing_uri = missing_uri.tolist() #convert to list

# #spotipy credentials
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials

# client_credentials_manager = SpotifyClientCredentials(client_id='fc03c036ef2e4552810bcfa717b8c9d6',
#                                                       client_secret='3f2bc1d385394fdebbfbca47a9b07268')
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# # sp.audio
# sp.audio_features('spotify:track:5hq4pLTfUbgJQ2NAPcIJHr')

# #query missing track info from api
# import pickle
# import time

# start = 0
# end = len(missing_uri)
# for i in range(start, end, 50):
#     print(i)
#     temp_list = missing_uri[i:i+50]
#     playlists = sp.audio_features(temp_list)

#     # read in existing file
# #     with open ('audio_features.txt', 'rb') as fp:
# #         itemlist = pickle.load(fp)
#     if "itemlist" in locals():
#         itemlist = itemlist + playlists # append new datat
#     else:
#         itemlist = playlists
    
#     if len(itemlist) == 600: # save partial data into 600 chunks in case of error
#         temp_filename = "missing_10k_" + str(i) + ".txt" 

#         with open(temp_filename,"wb") as fp: # write updated new data
#             pickle.dump(itemlist, fp)
#             del itemlist
#     time.sleep(np.random.randint(2,5))

In [16]:
#process queried data
import pickle
from collections import defaultdict
import csv
import numpy as np
import os

directory = 'missing_feature/' #get directory of artist data
extract = defaultdict(list)   
for file in os.listdir(directory): #for each txt file in folder
    if '.txt' in file:
        with open (directory+file, 'rb') as fp:
            itemlist = pickle.load(fp)
            try:
                for audiofile in itemlist:
                    if audiofile is None:
                        extract = extract
                    else:
                        for key in audiofile:
                            extract[str(key)].append(audiofile[key]) #append to dictionary
            except:
                print(file)

In [17]:
#missing track data convert to df
df_missing = pd.DataFrame.from_dict(extract)

In [18]:
df_missing = df_missing.drop(labels = ['track_href', 'analysis_url', 'type'], axis = 1) #drop unuseful columns
df_missing.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,uri,duration_ms,time_signature
0,0.614,0.796,1,-4.918,1,0.0417,0.000695,0.000002,0.0660,0.556,93.936,39rJM0PvYhYheKdoPj7mSs,spotify:track:39rJM0PvYhYheKdoPj7mSs,209107,4
1,0.627,0.560,5,-5.378,1,0.0667,0.455000,0.000000,0.1260,0.493,97.967,25tjqTcyrBqt6IHAbUJCYW,spotify:track:25tjqTcyrBqt6IHAbUJCYW,262853,4
2,0.736,0.681,5,-5.856,0,0.1010,0.126000,0.000000,0.3960,0.166,135.118,6m60IwLIkZ2Fmz4J9FsQ1o,spotify:track:6m60IwLIkZ2Fmz4J9FsQ1o,217507,4
3,0.700,0.915,9,-4.336,0,0.0530,0.040300,0.000019,0.3350,0.556,111.999,0eqX5nyQs6LkhbqpNVfTlm,spotify:track:0eqX5nyQs6LkhbqpNVfTlm,264643,4
4,0.492,0.727,11,-6.407,0,0.0481,0.022900,0.000056,0.0661,0.254,171.956,35FswkiJQH4qnbUAXI5RSJ,spotify:track:35FswkiJQH4qnbUAXI5RSJ,264400,4


In [19]:
df_missing = df_missing.rename(columns = {'uri': 'Track_uri'}) #rename cols for merging
# cols for merging
join_cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
             'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
             'Track_uri', 'duration_ms', 'time_signature']

In [20]:
subset10k.shape #check shape for comparison below

(679054, 22)

In [21]:
subset_10k_new = subset10k.merge(df_missing, on = 'Track_uri', how = 'left') #joining missing data

subset_10k_new = subset_10k_new.drop_duplicates() #drop duplicates

In [22]:
subset_10k_new.shape #check shape

(679054, 36)

In [23]:
subset_10k_new.columns

Index(['Playlistid', 'Trackid', 'Artist_Name', 'Track_Name', 'Album_Name',
       'Track_Duration', 'Artist_uri', 'Track_uri', 'Album_uri',
       'danceability_x', 'energy_x', 'key_x', 'loudness_x', 'mode_x',
       'speechiness_x', 'acousticness_x', 'instrumentalness_x', 'liveness_x',
       'valence_x', 'tempo_x', 'duration_ms_x', 'time_signature_x',
       'danceability_y', 'energy_y', 'key_y', 'loudness_y', 'mode_y',
       'speechiness_y', 'acousticness_y', 'instrumentalness_y', 'liveness_y',
       'valence_y', 'tempo_y', 'id', 'duration_ms_y', 'time_signature_y'],
      dtype='object')

In [ ]:
s = np.nan
a = 0.0555
np.isnan(a)

In [ ]:
def findVal(x):
    if np.isnan(x[0]) == True:
        return x[1]
    else:
        return x[0]

x_cols = ['danceability_x', 'energy_x', 'key_x', 'loudness_x', 'mode_x',
       'speechiness_x', 'acousticness_x', 'instrumentalness_x', 'liveness_x',
       'valence_x', 'tempo_x', 'duration_ms_x', 'time_signature_x']
y_cols = ['danceability_y', 'energy_y', 'key_y', 'loudness_y', 'mode_y',
       'speechiness_y', 'acousticness_y', 'instrumentalness_y', 'liveness_y',
       'valence_y', 'tempo_y', 'duration_ms_y', 'time_signature_y']
          
for i in range(len(x_cols)):
          name = x_cols[i][:-2]
          subset_10k_new[name] = subset_10k_new[[x_cols[i],y_cols[i]]].apply(lambda x: findVal(x), axis = 1)

In [ ]:
subset_10k_new = subset_10k_new.drop(x_cols, axis=1).drop(y_cols, axis=1)
subset_10k_new.head()

In [ ]:
subset_10k_new = subset_10k_new.drop('id', axis = 1)

In [ ]:
subset_10k_new.isna().sum()

In [ ]:
directory = 'missing_feature_2/' #get directory of artist data
extract2 = defaultdict(list)   
for file in os.listdir(directory): #for each txt file in folder
    if '.txt' in file:
        with open (directory+file, 'rb') as fp:
            itemlist = pickle.load(fp)
            try:
                for audiofile in itemlist:
                    if audiofile is None:
                        extract2 = extract2
                    else:
                        for key in audiofile:
                            extract2[str(key)].append(audiofile[key]) #append to dictionary
            except:
                print(file)

In [ ]:
#missing track data convert to df
df_missing2 = pd.DataFrame.from_dict(extract2)
df_missing2 = df_missing2.drop(labels = ['track_href', 'analysis_url', 'type'], axis = 1) #drop unuseful columns
df_missing2 = df_missing2.rename(columns = {'uri': 'Track_uri'}) #rename cols for merging
# cols for merging
join_cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
             'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
             'Track_uri', 'duration_ms', 'time_signature']


In [ ]:
subset_10k_new2 = subset_10k_new.merge(df_missing2, on = 'Track_uri', how = 'left') #joining missing data

subset_10k_new2 = subset_10k_new2.drop_duplicates() #drop duplicates

In [ ]:
subset_10k_new2.shape

In [ ]:
def findVal(x):
    if np.isnan(x[0]) == True:
        return x[1]
    else:
        return x[0]

x_cols = ['danceability_x', 'energy_x', 'key_x', 'loudness_x', 'mode_x',
       'speechiness_x', 'acousticness_x', 'instrumentalness_x', 'liveness_x',
       'valence_x', 'tempo_x', 'duration_ms_x', 'time_signature_x']
y_cols = ['danceability_y', 'energy_y', 'key_y', 'loudness_y', 'mode_y',
       'speechiness_y', 'acousticness_y', 'instrumentalness_y', 'liveness_y',
       'valence_y', 'tempo_y', 'duration_ms_y', 'time_signature_y']
          
for i in range(len(x_cols)):
          name = x_cols[i][:-2]
          subset_10k_new2[name] = subset_10k_new2[[x_cols[i],y_cols[i]]].apply(lambda x: findVal(x), axis = 1)

In [ ]:
subset_10k_new2 = subset_10k_new2.drop(x_cols, axis=1).drop(y_cols, axis=1)
subset_10k_new2 = subset_10k_new2.drop('id', axis = 1)
subset_10k_new2.isna().sum()

In [ ]:
incomplete = subset_10k_new2[subset_10k_new2.danceability.isna()]
complete = subset_10k_new2.loc[~subset_10k_new2.Playlistid.isin(incomplete.Playlistid)]

In [ ]:
len(complete.Playlistid.unique())

In [ ]:
final_data = complete.to_csv('../raw_data/final_data.csv')

In [ ]:
complete.shape

In [ ]:
complete_wa = complete.merge(artist_info, on = 'Artist_uri', how = 'left')

In [ ]:
complete_wa.shape

In [ ]:
complete_wa.isna().sum()

In [ ]:
missing_artist = complete_wa[complete_wa.popularity.isna()]

In [ ]:
len(missing_artist.Artist_uri.unique())

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id='fc03c036ef2e4552810bcfa717b8c9d6',
                                                      client_secret='3f2bc1d385394fdebbfbca47a9b07268')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
new_artist = list()
for p in missing_artist.Artist_uri.unique().tolist():
    new_artist.append(sp.artist(p))

In [ ]:
new_artist

In [ ]:
extract_artist = defaultdict(list)
for artist in new_artist:
    uri = artist['uri'] #get artist uri
    genres = artist['genres'] #get genres
    pop = artist['popularity'] #get popularity (on a scale of 1-100)
    fan = artist['followers']['total'] #get total number of followers  

    #append to dictionary
    extract_artist['Artist_uri'].append(uri)
    if len(genres) == 0:
        extract_artist['genres'].append(None)
    else:
        extract_artist['genres'].append(genres)
    extract_artist['popularity'].append(pop)
    extract_artist['artist_followers'].append(fan)

In [ ]:
df_new_artist = pd.DataFrame.from_dict(extract_artist)

In [ ]:
complete_wa = complete_wa.merge(df_new_artist, on = 'Artist_uri')

In [ ]:
complete_wa.columns

In [ ]:
def findVal(x):
    if np.isnan(x[0]) == True:
        return x[1]
    else:
        return x[0]

x_cols = ['popularity_x','artist_followers_x']
y_cols = ['popularity_y', 'artist_followers_y']
          
for i in range(len(x_cols)):
          name = x_cols[i][:-2]
          complete_wa[name] = complete_wa[[x_cols[i],y_cols[i]]].apply(lambda x: findVal(x), axis = 1)

In [ ]:
complete_wa = complete_wa.drop(x_cols, axis = 1).drop(y_cols, axis = 1)

In [ ]:
test = complete_wa[complete_wa.genres_x.isnull()]

In [ ]:
def findGenre(x):
    try:
        if np.isnan(x[0]) == True:
            return x[1]
        else:
            return x[0]
    except:
        return x[0]
complete_wa['genre'] = complete_wa[['genres_x','genres_y']].apply(lambda x: findGenre(x), axis = 1)

In [ ]:
complete_wa = complete_wa.drop(['genres_x','genres_y'], axis = 1)

In [ ]:
complete_wa.to_csv('../raw_data/final_data.csv')